In [3]:
import numpy as np
import tensorflow as tf
from vgg19_v26 import vgg19
import matplotlib.pyplot as plt
import os
from PIL import Image
from PIL import ImageEnhance
from tqdm import tqdm

In [4]:
img_H,img_W,img_C = 500, 800, 3
vgg19net = vgg19(trainable=False)
img = tf.get_variable(name='wanted', shape=(1,img_H,img_W,img_C))
vgg19net.inference(img, 1000)
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [8]:
# for i,v in enumerate(tf.global_variables()):
#     print('trainable:{0} {1:>2}'.format(v.trainable,i),':',v)

In [10]:
# for i,v in enumerate(tf.trainable_variables()):
#     print('trainable:{0} {1:>2}'.format(v.trainable,i),':',v)

In [9]:
# for i,v in enumerate(vgg19net.VARIABLE_list):
#     print('trainable:{0} {1:>2}'.format(v.trainable,i),':',v)

In [3]:
#     with open('./weight/all_name_list.txt', 'r') as flie:
#         all_list = flie.readlines()
#     all_list = list(map(lambda x: x[0:-1:1]+'.npy', all_list))
#     [len(all_list), len(vgg19net.VARIABLE_list)]
def read_pretrained_weight():
    for i in tqdm(range(32)):
        sess.run(vgg19net.VARIABLE_list[i].assign(np.load('./weight/' + f'keras_{i}.npy')))

In [4]:
# only run once
read_pretrained_weight()

100%|██████████████████████████████████████████████████████████████████████████████████| 32/32 [00:07<00:00,  2.23it/s]


In [5]:
def check_see():
    plt.figure(figsize=(20,10))
    for i in range(12):
        plt.subplot(3,4,i+1)
        see = sess.run(vgg19net.TENSOR_list[i])[0,:,:,32]
        plt.imshow((see-see.min())/(see.max()-see.min()), cmap='gray')
    plt.show()

def load_img(path):
    if path.split('.')[-1] == 'npy':
        return np.load(path)
    img    = Image.open(path).resize((img_W,img_H))
    img_np = np.zeros((1,img_H,img_W,img_C),dtype=np.float32)
    img_np[0,:,:,:] = img
    img_np = (img_np-img_np.min())/(img_np.max()-img_np.min())
    img_np = img_np - img_np.mean()
    return img_np

def save_img(path, value):
    value = value[0]
    value = (value - value.min())/(value.max() - value.min())
    value = Image.fromarray((value * 255).astype(np.uint8))
    value = ImageEnhance.Color(value).enhance(1.15)
    value = ImageEnhance.Contrast(value).enhance(1.15)
    value = ImageEnhance.Brightness(value).enhance(1.0)
    value.save(path)

def loss_content_func(sess, content_tensor_list):
    def loss_single_conv(fixed_content, tensor):
        num_C  = fixed_content.shape[3]
        num_HW = fixed_content.shape[1]*fixed_content.shape[2]
        return (1/(num_C*num_HW))*tf.reduce_sum(tf.pow(fixed_content-tensor,2))
    return sum([w*loss_single_conv(sess.run(tensor), tensor) for w, tensor in content_tensor_list])

def loss_style_func(sess, style_tensor_list):
    def gram_matrix(F, num_HW, num_C):
        F_ik = tf.reshape(F, (num_HW, num_C))
        return tf.matmul(tf.transpose(F_ik), F_ik)
    def loss_single_conv(fixed_content, tensor):
        num_C  = fixed_content.shape[3]
        num_HW = fixed_content.shape[1]*fixed_content.shape[2]
        fixed_content_gram = gram_matrix(fixed_content, num_HW, num_C)
        tensor_gram = gram_matrix(tensor, num_HW, num_C)
        return (1/(4*num_C*num_C*num_HW*num_HW))*tf.reduce_sum(tf.pow(fixed_content_gram-tensor_gram,2))
    return sum([w*loss_single_conv(sess.run(tensor), tensor) for w, tensor in style_tensor_list])

In [212]:
# img_content_np = load_img('content1.jpg')
# _ = sess.run(img.assign(img_content_np))
# check_see()
# img_style_np = load_img('style5.jpg')
# _ = sess.run(img.assign(img_style_np))
# check_see()

In [6]:
for i,o in enumerate(vgg19net.TENSOR_list):
    print('{0:>2}'.format(i),':',o)

 0 : Tensor("conv1_0/Relu:0", shape=(1, 500, 800, 64), dtype=float32)
 1 : Tensor("conv1_1/Relu:0", shape=(1, 500, 800, 64), dtype=float32)
 2 : Tensor("conv2_0/Relu:0", shape=(1, 250, 400, 128), dtype=float32)
 3 : Tensor("conv2_1/Relu:0", shape=(1, 250, 400, 128), dtype=float32)
 4 : Tensor("conv3_0/Relu:0", shape=(1, 125, 200, 256), dtype=float32)
 5 : Tensor("conv3_1/Relu:0", shape=(1, 125, 200, 256), dtype=float32)
 6 : Tensor("conv3_2/Relu:0", shape=(1, 125, 200, 256), dtype=float32)
 7 : Tensor("conv3_3/Relu:0", shape=(1, 125, 200, 256), dtype=float32)
 8 : Tensor("conv4_0/Relu:0", shape=(1, 63, 100, 512), dtype=float32)
 9 : Tensor("conv4_1/Relu:0", shape=(1, 63, 100, 512), dtype=float32)
10 : Tensor("conv4_2/Relu:0", shape=(1, 63, 100, 512), dtype=float32)
11 : Tensor("conv4_3/Relu:0", shape=(1, 63, 100, 512), dtype=float32)
12 : Tensor("conv5_0/Relu:0", shape=(1, 32, 50, 512), dtype=float32)
13 : Tensor("conv5_1/Relu:0", shape=(1, 32, 50, 512), dtype=float32)
14 : Tensor("con

# training: